In [1]:
!git clone https://github.com/MockaWolke/featout_hackathon
%cd featout_hackathon
!bash load_data.sh
!pip install -e .

Cloning into 'featout_hackathon'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 115 (delta 6), reused 21 (delta 6), pack-reused 91
Receiving objects: 100% (115/115), 28.81 MiB | 21.88 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/featout_hackathon
--2023-09-14 15:16:28--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.55.248.239, 2600:1406:3400:799::317f, 2600:1406:3400:785::317f, ...
Connecting to download.microsoft.com (download.microsoft.com)|23.55.248.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M  87.9MB/s    in 6.8s    

2023-09-14 15:16:35 (116 MB/s) - ‘kagglecatsanddog

In [2]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import csv
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from featout_exp.utils import load_data, load_model

In [3]:
BATCHSIZE = 32
NUM_EPOCHS = 10
EXP_NAME = "Control"
TESTING = False

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train_loader = load_data("train", 32, True)
test_loader = load_data("test", 32, False)
EXP_PATH = os.path.join("logs",EXP_NAME)

os.makedirs(EXP_PATH, exist_ok=True)
# Initialize TensorBoard writer
writer = SummaryWriter()

# Prepare a CSV file for logging
csv_file = open(os.path.join(EXP_PATH, "logs.csv"), "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Test Loss", "Test Accuracy"])

57

In [5]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    for step, (inputs, labels) in enumerate(bar):

        if TESTING == True and step > 2:
            break

        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = corrects.double() / total

    # Evaluate on test data
    model.eval()
    test_running_loss = 0.0
    test_corrects = 0
    test_total = 0

    with torch.no_grad():
        for step,  (inputs, labels) in enumerate(tqdm( test_loader)):
            if TESTING == True and step > 2:
                 break

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)
            test_total += labels.size(0)

    test_loss = test_running_loss / test_total
    test_acc = test_corrects.double() / test_total

    # Log metrics to TensorBoard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Loss/test", test_loss, epoch)
    writer.add_scalar("Accuracy/test", test_acc, epoch)

    # Log metrics to CSV
    csv_writer.writerow([epoch + 1, train_loss, train_acc.item(), test_loss, test_acc.item()])
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")

csv_file.close()
writer.close()

torch.save(model.state_dict(), os.path.join(EXP_PATH, "model.pth"))


  1%|▏         | 2/157 [00:00<00:14, 10.41it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 157/157 [00:16<00:00,  9.68it/s]


Epoch 1/10 - Train Loss: 0.5059 Train Acc: 0.7506 - Test Loss: 1.2963 Test Acc: 0.4308


100%|██████████| 157/157 [00:16<00:00,  9.41it/s]


Epoch 2/10 - Train Loss: 0.3510 Train Acc: 0.8434 - Test Loss: 1.9843 Test Acc: 0.4818


100%|██████████| 157/157 [00:17<00:00,  9.14it/s]


...

Epoch 8/10 - Train Loss: 0.1310 Train Acc: 0.9472 - Test Loss: 1.4537 Test Acc: 0.6180


100%|██████████| 157/157 [00:16<00:00,  9.31it/s]


Epoch 9/10 - Train Loss: 0.1136 Train Acc: 0.9545 - Test Loss: 1.3316 Test Acc: 0.6352


100%|██████████| 157/157 [00:16<00:00,  9.59it/s]

Epoch 10/10 - Train Loss: 0.0968 Train Acc: 0.9613 - Test Loss: 1.2496 Test Acc: 0.6556


In [6]:
!zip -r Control.zip logs/Control

  adding: logs/Control/ (stored 0%)
  adding: logs/Control/logs.csv (deflated 51%)
  adding: logs/Control/model.pth (deflated 8%)
